<a href="https://colab.research.google.com/github/Moly-malibu/DS-Unit-4-Sprint-1-NLP/blob/master/LS_DS_422_BOW_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Vector Representations
## *Data Science Unit 4 Sprint 2 Assignment 2*

In [0]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy

from bs4 import BeautifulSoup

## 1) *Clean:* Job Listings from indeed.com that contain the title "Data Scientist" 

You have `job_listings.csv` in the data folder for this module. The text data in the description column is still messy - full of html tags. Use the [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) library to clean up this column. You will need to read through the documentation to accomplish this task. 

In [126]:
from bs4 import BeautifulSoup
import requests

df = pd.read_csv('job_listings.csv')
df.head()

,Unnamed: 0,description,title
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist


## 2) Use Spacy to tokenize the listings 

In [0]:
import re
from nltk.corpus import stopwords
import string

In [0]:
#https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
#https://docs.python.org/3.1/library/re.html
#re_sub: Return the string obtained by replacing the leftmost non-overlapping 
#        occurrences of pattern in string by the replacement repl.
#https://www.programcreek.com/python/example/1275/string.punctuation
#https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

def word_tokenize(doc):
  table = str.maketrans(' ', ' ', string.punctuation)
  stop_words = set(stopwords.words('english'))
  
  #strip HTML:
  doc = re.sub('<[^<]+?>', '', doc) 

  #tokenize by word:
  tokens = word_tokenize(doc)

  #Punctuation:
  tokens = [x.lower().translate(table)
           for x in token]
  tokens = [word for word in tokens if word.isalpha()]

  #Remove stopwords:
  token = [w for w in tokens if not w in stop_words]

  #Lemmatize
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.Lemmatize(w) for w in tokens]

  return tokens

In [0]:
#clean DF
def clean_description(desc):
  soup = BeautifulSoup(desc)
  return soup.get_text()
df['clean'] = df['description'].apply(clean_description)

In [130]:
df.head()

,Unnamed: 0,description,title,clean
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist,"b""Job Requirements:\nConceptual understanding ..."
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I,"b'Job Description\n\nAs a Data Scientist 1, yo..."
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level,b'As a Data Scientist you will be working on c...
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist,"b'$4,969 - $6,756 a monthContractUnder the gen..."
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist,b'Location: USA \xe2\x80\x93 multiple location...


In [0]:
#token text
from spacy.tokenizer import Tokenizer
nlp = spacy.load('en_core_web_sm')

tokenizer = Tokenizer(nlp.vocab)

In [0]:
#tokenizer Pipe
tokens = []

for doc in tokenizer.pipe(df['clean'], batch_size=500):
  doc_token = []

  for token in doc:
    if (token.is_stop == False) & (token.is_punct == False):
      doc_token.append(token.text.lower())
  tokens.append(doc_token)

df['tokens'] = tokens

In [133]:
df.tail()

,Unnamed: 0,description,title,clean,tokens
421,421,"b""<b>About Us:</b><br/>\nWant to be part of a ...",Senior Data Science Engineer,"b""About Us:\nWant to be part of a fantastic an...","[b""about, us:\nwant, fantastic, fun, startup, ..."
422,422,"b'<div class=""jobsearch-JobMetadataHeader icl-...",2019 PhD Data Scientist Internship - Forecasti...,"b'InternshipAt Uber, we ignite opportunity by ...","[b'internshipat, uber,, ignite, opportunity, s..."
423,423,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist - Insurance,"b'$200,000 - $350,000 a yearA million people a...","[b'$200,000, $350,000, yeara, million, people,..."
424,424,"b""<p></p><div><p>SENIOR DATA SCIENTIST</p><p>\...",Senior Data Scientist,"b""SENIOR DATA SCIENTIST\nJOB DESCRIPTION\n\nAB...","[b""senior, data, scientist\njob, description\n..."
425,425,b'<div></div><div><div><div><div><p>Cerner Int...,Data Scientist,"b'Cerner Intelligence is a new, innovative org...","[b'cerner, intelligence, new,, innovative, org..."


## 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [134]:
#create Data Clean desc
data = df['clean']
data

0      b"Job Requirements:\nConceptual understanding ...
1      b'Job Description\n\nAs a Data Scientist 1, yo...
2      b'As a Data Scientist you will be working on c...
3      b'$4,969 - $6,756 a monthContractUnder the gen...
4      b'Location: USA \xe2\x80\x93 multiple location...
                             ...                        
421    b"About Us:\nWant to be part of a fantastic an...
422    b'InternshipAt Uber, we ignite opportunity by ...
423    b'$200,000 - $350,000 a yearA million people a...
424    b"SENIOR DATA SCIENTIST\nJOB DESCRIPTION\n\nAB...
425    b'Cerner Intelligence is a new, innovative org...
Name: clean, Length: 426, dtype: object

In [135]:
#Count Vectorizer 
vector = CountVectorizer(stop_words='english')

# build vocab
vector.fit(data)

# transform text
dtm = vector.transform(data)

type(dtm.todense())

numpy.matrix

In [136]:
#matrix
dtm.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 2, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]])

In [137]:
#features name, Get Word Counts for each document
dtm = pd.DataFrame(dtm.todense(), columns=vector.get_feature_names())
dtm.head()

,00,000,02115,03,0356,04,062,06366,08,10,100,1000,1001,1008,100k,100x,105,1079302,1082692,10b,10ms,10x,11,110k,112,114157802,12,120,125,126,130,1315,1324b,138,14,140,15,150,15000,159m,...,xbf,xbox,xc2,xc3,xe2,xef,xgboost,xp,xpo,yards,year,yeara,yearas,yearcollects,yeardescription,yearjob,yearlrs,years,yearsexperience,yearsummary,yearthe,yeartitle,yearworking,yes,yeti,yield,york,young,youtube,yrs,zenreach,zero,zeus,zf,zheng,zillow,zones,zoom,zuckerberg,zurich
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 4) Visualize the most common word counts

In [138]:
sum(dtm['zf'])

16

In [0]:
#Attributes
result = []
for col in dtm:
  x = sum(dtm[col])
  result.append(x)

data = zip(vector.get_feature_names(), result)
words = pd.DataFrame(data, columns=['words', 'count'])

In [140]:
words['rank'] = words['count'].rank(method='first',ascending=False)
words.tail()

,words,count,rank
9811,zillow,7,2665.0
9812,zones,1,9815.0
9813,zoom,1,9816.0
9814,zuckerberg,2,6441.0
9815,zurich,2,6442.0


In [141]:
words[words['rank']<=11]


,words,count,rank
539,analytics,730,10.0
1116,business,1198,5.0
2018,data,4394,1.0
2885,experience,1238,4.0
4389,learning,912,9.0
4584,machine,686,11.0
8063,science,956,8.0
8877,team,972,7.0
9629,work,976,6.0
9663,x80,1404,3.0


## 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [143]:
from sklearn.feature_extraction.text import TfidfVectorizer

data = df['clean']

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Create a vocabulary and get word counts per document
# Similiar to fit_predict
dtm = tfidf.fit_transform(data)

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head()

,000,04,10,100,1079302,11,12,125,14,15,150,17,18,180,19,1strategy,20,200,2012,2013,2015,2017,2018,2019,2020,24,25,30,300,30328,35,36,3rd,3x,40,401,401k,42,43,45,...,x94and,x94including,x94to,x94we,x98,x98big,x99,x99d,x99ll,x99re,x99s,x99t,x99ve,x9c,x9cbig,x9d,xa0,xa2,xa6,xa7,xac,xae,xb7,xbb,xc2,xc3,xe2,xef,xgboost,year,years,yearthe,yes,yeti,york,young,yrs,zeus,zf,zillow
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.138803,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.073933,0.0,0.0,0.079896,0.041748,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.108817,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.141143,0.0,0.0,0.041108,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.058492,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.086129,0.0,0.0,0.000000,0.093431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 6) Create a NearestNeighbor Model. Write the description of your ideal datascience job and query your job listings. 

In [144]:
from sklearn.neighbors import NearestNeighbors

#Fit on DTM

nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [145]:
nn.kneighbors([dtm.iloc[0].values])

(array([[0.        , 1.28186048, 1.2923534 , 1.29554001, 1.30410844]]),
 array([[  0, 336, 276, 274, 366]]))

In [146]:
#Query using kneighbors

nn.kneighbors([dtm.iloc[276]])

(array([[0.        , 1.02390365, 1.05990084, 1.20610327, 1.23749072]]),
 array([[276, 301, 392, 327, 399]]))

In [147]:
data[276][:200]

'b"Facebook\'s mission is to give people the power to build community and bring the world closer together. Through our family of apps and services, we\'re building a different kind of company that connec'

In [148]:
data[150]

"b'Contract1 year+ contract\\nData Scientist \\xe2\\x80\\x93 contract\\nEarly Clinical Development (ECD) department is seeking a Data Scientist reporting to the Predictive Analytics (gPA) Team Lead. The PA group supports clinical development in the area of trial design, study planning and the creation and application of predictive algorithms. The role will require cross-functional interactions with Clinical Science and Clinical Operations. The Data Scientist will act as a consultant and be required to both establish and apply new machine learning methods to specific problem domains within those functions.\\nThe Data Scientist will primarily be responsible for the development and the deployment of machine learning methods in early clinical development with particular focus on the application to: protocol writing, clinical trial planning/forecasting, and study execution. In addition, the data scientist may work to apply methods from the ECD AI (Artificial Intelligence) team to specific p

## Stretch Goals

 - Try different visualizations for words and frequencies - what story do you want to tell with the data?
 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.
 - Create a labeled dataset - which jobs will you apply for? Train a model to select the jobs you are most likely to apply for. :) 